In [1]:
!pip install -U bitsandbytes transformers peft accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 24.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 99.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.6/336.6 kB 23.9 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.0
    Uninstalling transformers-4.47.0:
      Successfully uninstalled transformers-4.47.0
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.2.1
    Uninstalling accelerate-1.2.1:
      Successfully uninstalled accelerate-1.2.1


In [2]:
from huggingface_hub import login
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer,BitsAndBytesConfig
model = AutoModelForCausalLM.from_pretrained("Ayaman77/Llama-2-7b-hf-personabased", device_map="auto")
tokenizer = AutoTokenizer.from_pretrained("Ayaman77/Llama-2-7b-hf-personabased")

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


model.safetensors:   0%|          | 0.00/4.17G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/183 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/978 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.62M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
def predict(persona, conv):
    a = "### Person B has the following Persona information.\n"
    b = """### Instruct: Person A and Person B are now having a conversation. 
Following the conversation below, write a response that Person B would say based on the above Persona information. 
Please carefully consider the flow and context of the conversation below, and use Person B's Persona information appropriately to generate a response that you think is the most appropriate reply for Person B.\n"""
    c = "### Output:\n"
    s = a + persona + "\n\n" + b + conv + "\n\n" + c
    inputs = tokenizer(s, return_tensors="pt").to(device)
    inputs_length = len(inputs["input_ids"][0])
    with torch.inference_mode():
        output = model.generate(
    **inputs,
    max_new_tokens=200,                 
    eos_token_id=tokenizer.eos_token_id, 
    pad_token_id=tokenizer.pad_token_id, 
    num_beams=5,                        
    temperature=0.9,                    
    top_k=10,                           
    top_p=0.92,                        
)

    return tokenizer.decode(output[0][inputs_length:]).split("### End")[0]

In [7]:
persona = "I always loved sports and found a way to make it a part of my career. I played soccer growing up, and now I coach a local youth team. My dad was my biggest inspiration—he was a former athlete and taught me a lot about discipline. I used to work as a lifeguard at the community pool before I started my current job as a fitness trainer."
conv = """Persona A: I'm really excited to be matched with you!
Persona B: That's awesome! I love all kinds of sports, but soccer is my favorite.
Persona A: Soccer is great! I played it growing up and now I coach a youth team.
Persona B: That's amazing! I played soccer too, and it's always fun to see young players grow.
Persona A: Yeah, it’s really rewarding. I also enjoy going to the gym and staying active.
Persona B: Same here! Staying fit is so important. Do you have a favorite workout routine?
"""

predict(persona, conv)

'I love lifting weights and doing cardio.'